In [3]:
import pandas as pd
import numpy as np
import random
import datetime
from datetime import datetime

## Notes

In [24]:
max_hist = 4
hist_idx, row_idx = list(range(-max_hist + 1, 1)), list(range(0, max_hist))
print(hist_idx)
print(row_idx)

[-3, -2, -1, 0]
[0, 1, 2, 3]


In [20]:
range_columns = list(range(0, max_hist))

In [22]:
for a, b in zip(range_hist, range_columns):
    print(a, b)

-3 0
-2 1
-1 2
0 3


## Dataset preparation

### Load dataset

In [47]:
data = pd.read_parquet('dataset_warzone_kd_bigger.parquet.gzip')
display(data.head())
print(data.shape)

,utcStartSeconds,utcEndSeconds,matchID,duration,playlistName,version,gameType,playerCount,teamCount,rankedTeams,...,player.awards.seven,player.awards.eight,playerStats.objectiveBrLootChopperBoxOpen,playerStats.objectiveShieldDamage,player.awards.simultaneous_kill,playerStats.objectiveBrPerseusLockerDoorOpenEe,squad,lobbykd,pct_playerskd,map
0,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,1.13,82.5,rebirth
1,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,1.13,82.5,rebirth
2,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,1.13,82.5,rebirth
3,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,1.13,82.5,rebirth
4,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,1.13,82.5,rebirth


(55270, 154)


In [ ]:
data.head()

,utcStartSeconds,utcEndSeconds,matchID,duration,playlistName,version,gameType,playerCount,teamCount,rankedTeams,...,player.awards.seven,player.awards.eight,playerStats.objectiveBrLootChopperBoxOpen,playerStats.objectiveShieldDamage,player.awards.simultaneous_kill,playerStats.objectiveBrPerseusLockerDoorOpenEe,squad,lobbykd,pct_playerskd,map
0,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,1.13,82.5,rebirth
1,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,1.13,82.5,rebirth
2,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,1.13,82.5,rebirth
3,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,1.13,82.5,rebirth
4,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,1.13,82.5,rebirth


In [3]:
categorical_features = [
    'map',
    'squad'
]
data[categorical_features] = data[categorical_features].astype('category')

# rename 'lobbykd' as 'target', since we want to predict the avg k/d ratio of our game
data = data.rename(columns={'lobbykd':'target'})
data = data.reindex(columns = [col for col in data.columns if col != 'target'] + ['target'])
data.head(2)

,utcStartSeconds,utcEndSeconds,matchID,duration,playlistName,version,gameType,playerCount,teamCount,rankedTeams,...,player.awards.seven,player.awards.eight,playerStats.objectiveBrLootChopperBoxOpen,playerStats.objectiveShieldDamage,player.awards.simultaneous_kill,playerStats.objectiveBrPerseusLockerDoorOpenEe,squad,pct_playerskd,map,target
0,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,82.5,rebirth,1.13
1,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,82.5,rebirth,1.13


In [4]:
# types, missing values
data.info(max_cols=199, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55270 entries, 0 to 64457
Data columns (total 154 columns):
 #    Column                                                                 Non-Null Count  Dtype   
---   ------                                                                 --------------  -----   
 0    utcStartSeconds                                                        55270 non-null  float64 
 1    utcEndSeconds                                                          55270 non-null  float64 
 2    matchID                                                                55270 non-null  object  
 3    duration                                                               55270 non-null  float64 
 4    playlistName                                                           0 non-null      object  
 5    version                                                                55270 non-null  float64 
 6    gameType                                                            

## Exploratory data analysis

### tmp tries count unique etc

In [5]:
df = data.copy(deep=True)

In [ ]:
print(df['total_bill'].astype(float).skew())
print(df['total_bill'].astype(float).kurt())

In [6]:
df[[
    'matchID',
    'playerStats.kills',
    'player.awards.streak_5',
    'player.awards.double',
    'playerStats.headshots'
]].describe()

,playerStats.kills,player.awards.streak_5,player.awards.double,playerStats.headshots
count,55270.000000,12250.000000,8931.000000,55270.000000
mean,2.838321,439350.151837,224679.443735,0.610602
std,2.996422,166094.935469,238632.817406,0.981670
min,0.000000,85488.000000,0.000000,0.000000
25%,1.000000,305928.000000,0.000000,0.000000
50%,2.000000,425040.000000,157680.000000,0.000000
75%,4.000000,567192.000000,399912.000000,1.000000
max,29.000000,912336.000000,872064.000000,17.000000


In [7]:
df[[
    'matchID',
    'playerStats.kills',
    'player.awards.streak_5',
    'player.awards.double',
    'playerStats.headshots'
]].query('`playerStats.headshots` >= 1').head(5)

,matchID,playerStats.kills,player.awards.streak_5,player.awards.double,playerStats.headshots
1,3409667761123289245,16.0,382752.0,765888.0,2.0
5,3409667761123289245,4.0,268368.0,NaN,3.0
6,3409667761123289245,4.0,530448.0,NaN,3.0
9,3409667761123289245,1.0,NaN,NaN,1.0
12,3409667761123289245,3.0,NaN,NaN,1.0


In [8]:
df[['matchID', 'player.awards.streak_5']].groupby("matchID")[['player.awards.streak_5']].apply(lambda x: (x.notnull()).sum()/len(x)*100).head(5)

,player.awards.streak_5
matchID,
10006862765041148091,13.333333
10022431813918295442,15.217391
10082009794454927010,32.075472
10090996203127539604,8.000000
10134718954536800884,19.565217


In [9]:
df[['matchID', 'player.awards.streak_5']].query('matchID == "10082009794454927010"').head(5)

,matchID,player.awards.streak_5
59796,10082009794454927010,176592.0
59797,10082009794454927010,486336.0
59798,10082009794454927010,NaN
59799,10082009794454927010,NaN
59800,10082009794454927010,210864.0


### Quick overall shape/stats at a glance

The main specificity of our dataset is that each 'record' (match) is 'multi-dimensional' (several players with n stats/features attached): <br>
I.e. We want to predict the avg kills/deaths (kd) ratio of each match, but each match does not correspond to a single row (a single set of features), but rather to n rows / players playing in this match.<br>
One Warzone "Resurgence" match --identified with a matchID, can count up to 50 teams of 1 player (solo mode), or up to 57 players when mode is set to duos, trios, quads. Each player (a row) in a match is attached with a number of features (number of kills, deaths, assists, headshots, xp awards...).<br>

In [10]:
# 55k records (players) but 1170 unique matches
print(data.shape)
print(data.matchID.nunique())

(55270, 154)
1170


In [11]:
data.describe()

,utcStartSeconds,utcEndSeconds,duration,version,playerCount,teamCount,playerStats.kills,playerStats.medalXp,playerStats.matchXp,playerStats.scoreXp,...,player.awards.backstab,playerStats.objectiveMedalScoreKillSsRadarDrone,player.awards.seven,player.awards.eight,playerStats.objectiveBrLootChopperBoxOpen,playerStats.objectiveShieldDamage,player.awards.simultaneous_kill,playerStats.objectiveBrPerseusLockerDoorOpenEe,pct_playerskd,target
count,5.527000e+04,5.527000e+04,55270.000000,55270.0,55270.000000,55270.000000,55270.000000,55270.000000,55270.000000,55270.000000,...,0.0,1.0,4.0,2.0,6.0,2.000000,2.0,1.0,55270.000000,55270.000000
mean,1.662564e+09,1.662565e+09,822161.697123,1.0,47.415017,17.527447,2.838321,110.598969,3104.130342,4270.571522,...,NaN,2.0,0.0,0.0,1.0,2.000000,0.0,1.0,79.011476,1.140005
std,4.859948e+05,4.859972e+05,50424.444630,0.0,5.087140,10.220768,2.996422,211.820421,1783.029725,3539.880240,...,NaN,NaN,0.0,0.0,0.0,1.414214,0.0,NaN,9.189453,0.132615
min,1.661549e+09,1.661550e+09,534000.000000,1.0,30.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,NaN,2.0,0.0,0.0,1.0,1.000000,0.0,1.0,60.000000,0.800000
25%,1.662241e+09,1.662242e+09,797000.000000,1.0,42.000000,11.000000,1.000000,0.000000,1692.000000,1500.000000,...,NaN,2.0,0.0,0.0,1.0,1.500000,0.0,1.0,72.500000,1.040000
50%,1.662377e+09,1.662378e+09,824000.000000,1.0,50.000000,14.000000,2.000000,20.000000,3153.000000,3575.000000,...,NaN,2.0,0.0,0.0,1.0,2.000000,0.0,1.0,79.069767,1.140000
75%,1.662850e+09,1.662851e+09,851000.000000,1.0,52.000000,18.000000,4.000000,80.000000,4324.000000,6175.000000,...,NaN,2.0,0.0,0.0,1.0,2.500000,0.0,1.0,85.454545,1.230000
max,1.663732e+09,1.663733e+09,970000.000000,1.0,58.000000,50.000000,29.000000,3740.000000,10622.000000,72530.000000,...,NaN,2.0,0.0,0.0,1.0,3.000000,0.0,1.0,100.000000,1.670000


In [12]:
data.groupby(by='matchID')[['target', 'playerStats.kills', 'playerStats.deaths', 'playerStats.kdRatio', 'playerStats.damageDone', 'playerStats.timePlayed']].agg(['last', 'count', 'sum', 'mean', 'median'])

target                           playerStats.kills        \
                       last count    sum  mean median              last count   
matchID                                                                         
10006862765041148091   1.18    45  53.10  1.18   1.18               0.0    45   
10022431813918295442   1.16    46  53.36  1.16   1.16               2.0    46   
10082009794454927010   1.34    53  71.02  1.34   1.34               4.0    53   
10090996203127539604   0.99    50  49.50  0.99   0.99               0.0    50   
10134718954536800884   1.11    46  51.06  1.11   1.11               6.0    46   
...                     ...   ...    ...   ...    ...               ...   ...   
9952914128324669505    0.85    52  44.20  0.85   0.85               1.0    52   
9961080040106933532    1.17    40  46.80  1.17   1.17               2.0    40   
9963452121608988341    1.24    52  64.48  1.24   1.24               1.0    52   
9971642112346017660    1.08    40  43.20  1.08   1.08               2.0    40   
9976478907563874466    1.08    53  57.24  1.08   1.08               3.0    53   

                                              ... playerStats.damageDone  \
                        sum      mean median  ...                   last   
matchID                                       ...                          
10006862765041148091  101.0  2.244444    2.0  ...                  443.0   
10022431813918295442   86.0  1.869565    1.0  ...                  923.0   
10082009794454927010  185.0  3.490566    3.0  ...                 1356.0   
10090996203127539604   83.0  1.660000    1.0  ...                    3.0   
10134718954536800884  114.0  2.478261    2.0  ...                 1894.0   
...                     ...       ...    ...  ...                    ...   
9952914128324669505   163.0  3.134615    3.0  ...                 1059.0   
9961080040106933532   111.0  2.775000    2.0  ...                 1064.0   
9963452121608988341   140.0  2.692308    2.0  ...                  890.0   
9971642112346017660   115.0  2.875000    2.0  ...                 1341.0   
9976478907563874466   176.0  3.320755    3.0  ...                 2225.0   

                                                           \
                     count       sum         mean  median   
matchID                                                     
10006862765041148091    45   58445.0  1298.777778  1062.0   
10022431813918295442    46   44715.0   972.065217   799.5   
10082009794454927010    53  104978.0  1980.716981  1658.0   
10090996203127539604    50   49394.0   987.880000   715.0   
10134718954536800884    46   62215.0  1352.500000  1065.5   
...                    ...       ...          ...     ...   
9952914128324669505     52   94405.0  1815.480769  1552.0   
9961080040106933532     40   67647.0  1691.175000  1528.5   
9963452121608988341     52   80934.0  1556.423077  1341.5   
9971642112346017660     40   61750.0  1543.750000  1336.0   
9976478907563874466     53  101962.0  1923.811321  1874.0   

                     playerStats.timePlayed                                    
                                       last count      sum        mean median  
matchID                                                                        
10006862765041148091                  258.0    45  20270.0  450.444444  449.0  
10022431813918295442                  316.0    46  17601.0  382.630435  327.0  
10082009794454927010                  422.0    53  31395.0  592.358491  674.0  
10090996203127539604                  142.0    50  22176.0  443.520000  311.0  
10134718954536800884                  839.0    46  20768.0  451.478261  484.0  
...                                     ...   ...      ...         ...    ...  
9952914128324669505                   380.0    52  36619.0  704.211538  774.0  
9961080040106933532                   819.0    40  23387.0  584.675000  731.5  
9963452121608988341                   203.0    52  28570.0  549.423077  603.0  
997164

In [13]:
pd.DataFrame(data.isna().sum().div(len(data)).mul(100).sort_values(ascending=False))[0:50]

,0
playlistName,100.000000
rankedTeams,100.000000
player.awards.backstab,100.000000
playerStats.objectiveHack,99.998191
playerStats.objectiveMedalScoreKillSsRadarDrone,99.998191
playerStats.objectiveDestroyedVehicleMedium,99.998191
playerStats.objectiveTrophyDefense,99.998191
playerStats.objectiveBrPerseusLockerDoorOpenEe,99.998191
playerStats.objectiveBrForgottenLockerDoorOpenEe,99.998191
player.awards.simultaneous_kill,99.996381


In [14]:
pd.DataFrame(data.groupby(by='matchID').agg('mean').isna().sum().sort_values(ascending=False))[70:90]

,0
player.brMissionStats.missionStatsByType.masterassassination.xp,528
player.awards.triple,524
player.awards.explosive_stick,477
player.brMissionStats.missionStatsByType.timedrun.count,331
player.brMissionStats.missionStatsByType.timedrun.weaponXp,331
player.brMissionStats.missionStatsByType.timedrun.xp,331
player.awards.comeback,310
player.awards.air_to_air_kill,289
player.brMissionStats.missionStatsByType.scavenger.count,288
player.brMissionStats.missionStatsByType.scavenger.xp,288


##### small focus outlier on DamageDone

In [15]:
# notes:
# EDA : one extreme outlier (+ some suspects) on damageDone with max values way above average
# but we keep them as we will encounter them in real life + we're using xgboost

In [16]:
data[['matchID', 'playerCount', 'squad', 'map', 'playerStats.damageDone']].sort_values(by='playerStats.damageDone', ascending=False).head(20)

,matchID,playerCount,squad,map,playerStats.damageDone
38613,1370069958461487008,52.0,Quads,fortkeep,4.294967e+09
55090,855032827875055390,42.0,Quads,rebirth,1.271800e+04
63532,10628305472550279382,40.0,Quads,rebirth,1.239100e+04
10273,7362251540313581052,52.0,Quads,fortkeep,1.148700e+04
39966,3765989571523210640,44.0,Trios,rebirth,1.113800e+04
9986,1501974320764785975,40.0,Quads,rebirth,1.056500e+04
13144,13547399530503207787,52.0,Quads,fortkeep,1.053500e+04
50310,1766746682563678467,40.0,Quads,rebirth,1.052700e+04
49386,2449047677272433734,39.0,Quads,rebirth,1.047700e+04
46143,14089138655969584359,40.0,Quads,rebirth,1.026200e+04


## XGB Prediction

### Train Test Split

In [17]:
# As we have multidimensional data for each match (one match = several players/rows, make sure we train test split
# keeping all rows (players) with same match Id in respective train or test dataset
# we're keeping 20 pct of match Ids for test dataset

random.seed(42)
match_uuids = list(set(data.matchID.tolist()))
split_ids = random.sample(match_uuids, int(20*len(match_uuids)/100))

test = data.query('matchID in @split_ids')
train = data.query('matchID not in @split_ids')

display(train.head(2))
print(test.shape)
print(train.shape)

,utcStartSeconds,utcEndSeconds,matchID,duration,playlistName,version,gameType,playerCount,teamCount,rankedTeams,...,player.awards.seven,player.awards.eight,playerStats.objectiveBrLootChopperBoxOpen,playerStats.objectiveShieldDamage,player.awards.simultaneous_kill,playerStats.objectiveBrPerseusLockerDoorOpenEe,squad,pct_playerskd,map,target
0,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,82.5,rebirth,1.13
1,1.662835e+09,1.662836e+09,3409667761123289245,821000.0,None,1.0,wz,40.0,12.0,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Quads,82.5,rebirth,1.13


(10941, 154)
(44329, 154)


### Features engineering

#### features selection

In [18]:
# with lot of back and forth cv runs + features importance for feature selection : rmse : 0.1148 => 0.1136

In [19]:
def select_features(df):
    """
    Retains the best columns we will build features upon
    """
    to_keep = [
        "matchID",
        "utcEndSeconds",
        "map",
        "squad",
        "duration",
        "playerCount",
        "teamCount",
        "playerStats.kills",
        "playerStats.deaths",
        "playerStats.assists",
        "playerStats.scorePerMinute",
        "playerStats.headshots",
        "playerStats.rank",
        "playerStats.distanceTraveled",
        "playerStats.teamSurvivalTime",
        "playerStats.kdRatio",
        "playerStats.timePlayed",
        "playerStats.percentTimeMoving",
        "playerStats.damageDone",
        "playerStats.damageTaken",
        "player.awards.streak_5",
        "player.awards.double",
        "player.brMissionStats.missionsComplete",
    ]

    return df[to_keep]

#### encode time / categorical features

In [20]:
from sklearn.preprocessing import OneHotEncoder

def encode_features(df):
    """
    Encode datetime, categorical (squad size, map type) columns
    """

    def encode_datetime(df):
        """
        Add day of week, hour, from timestamp
        """

        df["utcEndSeconds"] = df["utcEndSeconds"].apply(
            lambda x: datetime.fromtimestamp(x)
        )
        df["weekday"] = df["utcEndSeconds"].dt.weekday
        df["hour"] = df["utcEndSeconds"].dt.hour
        # df.drop("utcEndSeconds", axis=1, inplace=True)

        return df

    def squad_to_ordinal(df):
        """
        label (ordinal) encoding for 'squad' (Solos, Duos, Trios...)
        (could also use one hot, but squad is kind of ordinal)
        """
        squad_order = {"Solos": 1, "Duos": 2, "Trios": 3, "Quads": 4}
        df["squad_ordinal"] = df["squad"].map(squad_order)
        df["squad_ordinal"] = df["squad_ordinal"].astype("int64")
        df.drop("squad", axis=1, inplace=True)

        return df

    def one_hot(df, column):
        """
        One Hot Encode one categorical column using sklearn
        ohe encoder previously fit when we built our model
        """

        with open("src/model/ohe_encoder.pickle", "rb") as f:
            enc = pickle.load(f)
        encoded_features = enc.transform(df[[column]]).toarray()

        df_features = pd.DataFrame(encoded_features)
        columns = enc.get_feature_names_out([column]).tolist()
        df_features.columns = columns

        for _ in [df, df_features]:
            _.reset_index(drop=True, inplace=True)
        augmented_df = pd.concat([df, df_features], axis=1)
        augmented_df.drop(column, axis=1, inplace=True)

        return augmented_df

    # apply encoding
    encoded_df = encode_datetime(df)
    encoded_df = squad_to_ordinal(encoded_df)
    encoded_df = one_hot(encoded_df, column="map")

    return encoded_df

#### feature creation

In [21]:
def create_new_features(df):
    """
    Create new features from existing features
    We calculated and tried a lot, but found those ones to work better
    """

    def add_time_slot(df):
        """
        Hour (0-24) had not much effect, let's custom-bin it
        morning (1), noon (2), afternoon (3), evening (4), late evening (5)
        """
        dict_ = {
            6:1, 7:1, 8:1, 9:1,
            10:1, 11:2, 12:2,
            13:2, 14:3, 15:3,
            16:3, 17:3, 18:4,
            19:4, 20:4, 21:4,
            22:5, 23:5, 0:5,
            1:5, 2:5, 3:5,
            4:5, 5:5,
        }
        df["time_slot"] = df["hour"].map(dict_)

        return df

    def normalize_by_time_played(df):
        """
        kills, damage... / time played
        """
        columns = [
            "playerStats.kills",
            "playerStats.deaths",
            "playerStats.damageDone",
            "playerStats.damageTaken",
        ]
        for col in columns:
            df[col + "_by_timePlayed"] = df[col].div(df["playerStats.timePlayed"])

        return df

    def damage_by_kill(df):
        """
        damageDone to get a kill
        """
        columns = [
            "playerStats.damageDone",
        ]
        for col in columns:
            df[col + "_by_kill"] = df[col].div(df["playerStats.kills"])

        return df

    def headshot_by_kill(df):
        """
        headshot / kill
        """
        columns = [
            "playerStats.headshots",
        ]
        for col in columns:
            # + .1 to prevent inf / nan values
            df[col + "_by_kill"] = df[col].add(0.1).div(df["playerStats.kills"] + 0.1)

        return df

    # apply features creation

    augmented_df = add_time_slot(df)
    augmented_df = normalize_by_time_played(augmented_df)
    augmented_df = damage_by_kill(augmented_df)
    augmented_df = headshot_by_kill(augmented_df)

    return augmented_df

#### features aggregation : keep one row (players' stats aggregation) per matchID

In [22]:
def perform_aggregations(df):
    """
    A match consist of +- 40 players (rows); we have only one single given target : "lobby kd"
    We aggregate players rows, sometimes adding new features, to keep one single array of features per match
    We tried others methods to aggregate (players placement, percentiles etc), but they did not add as much.
    """

    # those features won't be aggregated, because they're the same for all players
    no_agg_columns = [
        "utcEndSeconds",
        "duration",
        "playerCount",
        "teamCount",
        "weekday",
        "hour",
        "squad_ordinal",
        "map_fortkeep",
        "map_rebirth",
        "time_slot",
    ]

    # Features such as kills, deaths etc.. + newly created features are aggregated using, mean, std, median
    detailed_agg_columns = [
        "playerStats.rank",
        "playerStats.kdRatio",
        "playerStats.kills",
        "playerStats.deaths",
        "playerStats.assists",
        "playerStats.damageDone",
        "playerStats.damageTaken",
        "playerStats.kills_by_timePlayed",
        "playerStats.deaths_by_timePlayed",
        "playerStats.damageDone_by_timePlayed",
        "playerStats.damageTaken_by_timePlayed",
        "playerStats.damageDone_by_kill",
        "playerStats.scorePerMinute",
        "playerStats.teamSurvivalTime",
        "playerStats.timePlayed",
        "playerStats.percentTimeMoving",
        "player.awards.streak_5",
        "player.awards.double",
        "playerStats.headshots",
        "player.brMissionStats.missionsComplete",
        "playerStats.headshots_by_kill",
    ]

    # keep core features (do not vary per players) :

    df_core = df.groupby("matchID")[no_agg_columns].agg("last")

    # perform mean, std, median groupby-agg on other features:

    df_detailed = df.groupby("matchID")[detailed_agg_columns].agg(
        ["mean", "std", "median"]
    )
    df_detailed.columns = ["_".join(x) for x in df_detailed.columns]

    # perform special aggregations (count of given variables among players of a match),
    # adding new features :

    pct_players_0_kills = (
        df[["matchID", "playerStats.kills"]]
        .groupby("matchID")[["playerStats.kills"]]
        .apply(lambda x: (x == 0).sum())
    )
    pct_players_0_kills.columns = ["pct_players_0_kills"]
    pct_players_5_kills = (
        df[["matchID", "playerStats.kills"]]
        .groupby("matchID")[["playerStats.kills"]]
        .apply(lambda x: (x >= 5).sum())
    )
    pct_players_5_kills.columns = ["pct_players_5_kills"]
    pct_players_10_kills = (
        df[["matchID", "playerStats.kills"]]
        .groupby("matchID")[["playerStats.kills"]]
        .apply(lambda x: (x >= 10).sum())
    )
    pct_players_10_kills.columns = ["pct_players_10_kills"]

    pct_players_with_streak_5 = (
        df[["matchID", "player.awards.streak_5"]]
        .groupby("matchID")[["player.awards.streak_5"]]
        .apply(lambda x: (x.notnull()).sum() / len(x) * 100)
    )
    pct_players_with_streak_5.columns = ["pct_players_with_streak_5"]

    pct_players_with_double = (
        df[["matchID", "player.awards.double"]]
        .groupby("matchID")[["player.awards.double"]]
        .apply(lambda x: (x.notnull()).sum() / len(x) * 100)
    )
    pct_players_with_double.columns = ["pct_players_with_double"]

    pct_players_with_headshots = (
        df[["matchID", "playerStats.headshots"]]
        .groupby("matchID")[["playerStats.headshots"]]
        .apply(lambda x: (x.notnull()).sum() / len(x) * 100)
    )
    pct_players_with_headshots.columns = ["pct_players_with_headshots"]

    # concatenate all columns (features), along matchID index
    df = pd.concat(
        [
            df_core,
            df_detailed,
            pct_players_0_kills,
            pct_players_5_kills,
            pct_players_10_kills,
            pct_players_with_streak_5,
            pct_players_with_double,
            pct_players_with_headshots,
        ],
        axis=1,
    ).reset_index()

    return df

#### Complete Features engineering Pipeline 

In [ ]:
def pipeline_transform(last_session):
    """
    Apply all above functions to get our data ready for prediction
    """
    df = pd.read_parquet('dataset_warzone_kd_bigger.parquet.gzip')
    
    df = select_features(df)
    df = encode_features(df)
    df = create_new_features(df)
    df = perform_aggregations(df)

    df = df.drop(["matchID", "utcEndSeconds"], axis=1)
    return df

### XGBOOST

#### Apply features engineering to train test data

In [23]:
train = shrink_features(train)
train = encode_datetime(train)
train = add_time_slot(train)
train = squad_to_ordinal(train)
train = one_hot(train, column='map')
train = normalize_by_time_played(train)
train = damage_by_kill(train)
train = headshot_by_kill(train)
train = aggregate_players(train)

train = train.reset_index()
train.head(1)

final data shape : (936, 79)


,matchID,target,duration,playerCount,teamCount,weekday,hour,squad_ordinal,map_fortkeep,map_rebirth,...,player.brMissionStats.missionsComplete_median,playerStats.headshots_by_kill_mean,playerStats.headshots_by_kill_std,playerStats.headshots_by_kill_median,pct_players_0_kills,pct_players_5_kills,pct_players_10_kills,pct_players_with_streak_5,pct_players_with_double,pct_players_with_headshots
0,10022431813918295442,1.16,729000.0,46.0,24.0,0,2,2,0.0,1.0,...,0.0,0.372964,0.402746,0.090909,10,5,0,15.217391,19.565217,100.0


In [24]:
train.info(max_cols=199, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 936 entries, 0 to 935
Data columns (total 80 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   matchID                                        936 non-null    object 
 1   target                                         936 non-null    float64
 2   duration                                       936 non-null    float64
 3   playerCount                                    936 non-null    float64
 4   teamCount                                      936 non-null    float64
 5   weekday                                        936 non-null    int64  
 6   hour                                           936 non-null    int64  
 7   squad_ordinal                                  936 non-null    int64  
 8   map_fortkeep                                   936 non-null    float64
 9   map_rebirth                                    936 non

In [25]:
test = shrink_features(test)
test = encode_datetime(test)
test = add_time_slot(test)
test = squad_to_ordinal(test)
test = one_hot(test, column='map')
test = normalize_by_time_played(test)
test = damage_by_kill(test)
test = headshot_by_kill(test)
test = aggregate_players(test)

test = test.reset_index()
test.head(1)

final data shape : (234, 79)


,matchID,target,duration,playerCount,teamCount,weekday,hour,squad_ordinal,map_fortkeep,map_rebirth,...,player.brMissionStats.missionsComplete_median,playerStats.headshots_by_kill_mean,playerStats.headshots_by_kill_std,playerStats.headshots_by_kill_median,pct_players_0_kills,pct_players_5_kills,pct_players_10_kills,pct_players_with_streak_5,pct_players_with_double,pct_players_with_headshots
0,10006862765041148091,1.18,728000.0,45.0,16.0,6,20,3,0.0,1.0,...,0.0,0.646735,1.636913,0.180328,13,8,1,13.333333,20.0,100.0


#### Features / target split

In [28]:
X_train = train.drop(['matchID', 'target'], axis=1).copy()
y_train = train['target'].copy()
print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_test = test.drop(['matchID', 'target'], axis=1).copy()
y_test = test['target'].copy()
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")


X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)
print(f"full X: {X.shape}")
print(f"full y: {y.shape}")

X_train shape: (936, 78)
y_train shape: (936,)
X_test shape: (234, 78)
y_test shape: (234,)
full X: (1170, 78)
full y: (1170,)


#### Model with XGBoost

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [41]:
# Sklearn API, default run, no hyper params tuning (expect learning rate found to be 0.03 - 0.05)
# early stopping set to 50 to prevent overfitting

reg = xgb.XGBRegressor(n_estimators=1000, early_stopping_rounds=50, learning_rate=0.03)
reg.fit(
    X_train,
    y_train,
    verbose=10,
    eval_set=[(X_train, y_train), (X_test, y_test)]
)

[0]	validation_0-rmse:0.63240	validation_1-rmse:0.63708
[10]	validation_0-rmse:0.47316	validation_1-rmse:0.47972
[20]	validation_0-rmse:0.35654	validation_1-rmse:0.36583
[30]	validation_0-rmse:0.27089	validation_1-rmse:0.28564
[40]	validation_0-rmse:0.20786	validation_1-rmse:0.22951
[50]	validation_0-rmse:0.16175	validation_1-rmse:0.19047
[60]	validation_0-rmse:0.12823	validation_1-rmse:0.16484
[70]	validation_0-rmse:0.10392	validation_1-rmse:0.14785
[80]	validation_0-rmse:0.08636	validation_1-rmse:0.13742
[90]	validation_0-rmse:0.07370	validation_1-rmse:0.13107
[100]	validation_0-rmse:0.06447	validation_1-rmse:0.12728
[110]	validation_0-rmse:0.05772	validation_1-rmse:0.12506
[120]	validation_0-rmse:0.05266	validation_1-rmse:0.12342
[130]	validation_0-rmse:0.04874	validation_1-rmse:0.12277
[140]	validation_0-rmse:0.04581	validation_1-rmse:0.12235
[150]	validation_0-rmse:0.04342	validation_1-rmse:0.12201
[160]	validation_0-rmse:0.04142	validation_1-rmse:0.12173
[170]	validation_0-rmse:0

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=50, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.03, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

#### Features importance

In [ ]:
# using sklearn, default measure
pd.DataFrame(data=reg.feature_importances_,
             index=reg.feature_names_in_).sort_values(by=0, ascending=False).head(50)

In [ ]:
# using SHAP
import shap

explainer = shap.Explainer(reg)
shap_values = explainer(X)
shap.plots.waterfall(shap_values[0], max_display=25)

In [ ]:
shap.plots.beeswarm(shap_values)

In [ ]:
shap.plots.bar(shap_values, max_display=25)

#### Hyperparameters tuning

In [29]:
# Random grid search using RandomSearchCV() from Sklearn

from sklearn.model_selection import RandomizedSearchCV

# we already went into back and forth runs to run with below ranges :
params = {
    'learning_rate': [0.03, 0.031, 0.032, 0.033, 0.034],
    'max_depth': [3],
    'subsample': [0.6],
    'colsample_bytree': [0.7, 0.75, 0.77],
    'colsample_bylevel': [0.7, 0.75, 0.77],
    'n_estimators': [230, 240, 250, 280],
    'reg_alpha': [0, 0.5],
    'reg_lambda': [0, 0.5]
}

xgbr = xgb.XGBRegressor(seed = 20)
clf = RandomizedSearchCV(estimator=xgbr,
                         n_jobs= -1,
                         refit=True, #default would be True anyways
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=200,
                         verbose=1,
                         random_state=42)
clf.fit(X_train, y_train)
print("Lowest RMSE: ", (-clf.best_score_)**(1/2.0))
print("Best parameters:\n", clf.best_params_)

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Best parameters: {'subsample': 0.6, 'reg_lambda': 0, 'reg_alpha': 0, 'n_estimators': 240, 'max_depth': 3, 'learning_rate': 0.03, 'colsample_bytree': 0.75, 'colsample_bylevel': 0.75}
Lowest RMSE:  0.11022444437542547


In [30]:
best_pars = clf.best_params_
best_model = clf.best_estimator_

#### Final evaluation on Testing set

In [38]:
# when trained on the train dataset

best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

0.11694130019654217

In [39]:
# when trained on the union of train and validation sets

best_model.fit(X, y)
y_pred = best_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
rmse

0.08668848952302853

In [ ]:
# save best model with sklearn
# best_model.save_model('xgb_model_lobby_kd.json')

### Save / load / predict

In [42]:
# save
best_model.save_model('xgb_model_lobby_kd_2.json')

In [43]:
# load
model = xgb.XGBRegressor()
model.load_model("xgb_model_lobby_kd_2.json")

In [44]:
predictions = model.predict(X_test)
predictions

array([1.1298218 , 1.1598041 , 0.99106485, 1.1223193 , 1.0733162 ,
       1.2715098 , 1.2518163 , 0.9020997 , 1.1843241 , 1.1018157 ,
       1.035967  , 1.1389608 , 1.1518203 , 1.035422  , 1.2124293 ,
       0.9343143 , 1.1135195 , 1.2375487 , 1.1848292 , 1.1093271 ,
       1.1730161 , 1.1522185 , 1.2715495 , 1.149849  , 1.0782014 ,
       1.0913879 , 1.0353247 , 1.2026341 , 1.2638931 , 1.1537133 ,
       1.1619048 , 1.2526605 , 1.3509421 , 0.97241056, 1.0679954 ,
       1.0759621 , 1.1333247 , 1.1776549 , 1.1109833 , 1.15159   ,
       1.0923238 , 1.288236  , 1.2442045 , 1.1904253 , 1.0302621 ,
       1.1370406 , 1.2387992 , 0.9690744 , 1.1686134 , 1.2590287 ,
       1.1689124 , 1.1356472 , 1.1428266 , 1.1464666 , 1.0898745 ,
       1.3007495 , 1.1228284 , 1.1180295 , 1.034966  , 1.1398808 ,
       1.1352593 , 1.1558567 , 1.112502  , 1.1502775 , 1.1800678 ,
       1.1392734 , 1.1144649 , 1.0884413 , 1.1261197 , 1.160377  ,
       1.1314647 , 1.0978576 , 1.1397892 , 1.0527076 , 1.09249

In [45]:
full_prediction = X_test.copy(deep=True)
full_prediction['wzranked_kd'] = test['target']
full_prediction['predicted_kd'] = predictions
full_prediction

,duration,playerCount,teamCount,weekday,hour,squad_ordinal,map_fortkeep,map_rebirth,time_slot,playerStats.rank_mean,...,playerStats.headshots_by_kill_std,playerStats.headshots_by_kill_median,pct_players_0_kills,pct_players_5_kills,pct_players_10_kills,pct_players_with_streak_5,pct_players_with_double,pct_players_with_headshots,wzranked_kd,predicted_kd
0,728000.0,45.0,16.0,6,20,3,0.0,1.0,4,53.911111,...,1.636913,0.180328,13,8,1,13.333333,20.000000,100.0,1.18,1.129822
1,895000.0,40.0,10.0,6,0,4,0.0,1.0,5,53.575000,...,0.331580,0.192154,4,15,2,30.000000,15.000000,100.0,1.16,1.159804
2,814000.0,40.0,11.0,0,12,4,0.0,1.0,2,53.325000,...,0.332399,0.158065,5,10,1,32.500000,15.000000,100.0,0.98,0.991065
3,809000.0,52.0,15.0,6,21,4,1.0,0.0,4,52.750000,...,0.355714,0.304476,9,12,1,21.153846,9.615385,100.0,0.88,1.122319
4,794000.0,52.0,26.0,4,17,2,1.0,0.0,3,52.903846,...,0.422126,0.325307,18,8,0,17.307692,13.461538,100.0,1.03,1.073316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,740000.0,46.0,23.0,5,20,2,0.0,1.0,4,54.000000,...,0.419772,0.439324,21,5,1,10.869565,19.565217,100.0,1.29,1.175963
230,903000.0,52.0,13.0,0,12,4,1.0,0.0,2,53.173077,...,1.518689,0.249531,15,20,2,36.538462,21.153846,100.0,1.15,1.145893
231,864000.0,47.0,46.0,6,15,1,0.0,1.0,3,52.574468,...,0.397014,0.295775,12,13,2,27.659574,12.765957,100.0,1.31,1.209664
232,790000.0,43.0,15.0,0,17,3,0.0,1.0,3,53.441860,...,0.425750,0.268293,11,7,1,16.279070,16.279070,100.0,1.22,1.143624


In [ ]:
X_test.columns.tolist()

### misc Pipeline / export

#### export One hot encoder, for in production data-reshaping

In [ ]:
data = pd.read_parquet('dataset_warzone_kd_bigger.parquet.gzip')

In [ ]:
enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(data[['map']])
with open('ohe_encoder.pickle', 'wb') as f:
    pickle.dump(enc, f)